In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
%%writefile example.txt
first line
second line
thrid line
fourth line

Writing example.txt


In [4]:
textFile = sc.textFile('example.txt')

In [5]:
textFile.count()

4

In [6]:
textFile.first()

u'first line'

In [7]:
secf = textFile.filter(lambda line : 'second' in line)

In [9]:
secf

PythonRDD[4] at RDD at PythonRDD.scala:48

In [10]:
secf.collect()

[u'second line']

In [11]:
secf.count()

1

In [12]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

Writing example2.txt


In [13]:
text_rdd = sc.textFile('example2.txt')

In [14]:
words = text_rdd.map(lambda line : line.split())

In [16]:
words.collect()

[[u'first'],
 [u'second', u'line'],
 [u'the', u'third', u'line'],
 [u'then', u'a', u'fourth', u'line']]

In [17]:
text_rdd.collect()

[u'first', u'second line', u'the third line', u'then a fourth line']

In [18]:
text_rdd.flatMap(lambda line : line.split()).collect()

[u'first',
 u'second',
 u'line',
 u'the',
 u'third',
 u'line',
 u'then',
 u'a',
 u'fourth',
 u'line']

In [19]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [20]:
services = sc.textFile('services.txt')

In [21]:
services.take(2)

[u'#EventId    Timestamp    Customer   State    ServiceID    Amount',
 u'201       10/13/2017      100       NY       131          100.00']

In [22]:
services.map(lambda line : line.split()).take(3)

[[u'#EventId', u'Timestamp', u'Customer', u'State', u'ServiceID', u'Amount'],
 [u'201', u'10/13/2017', u'100', u'NY', u'131', u'100.00'],
 [u'204', u'10/18/2017', u'700', u'TX', u'129', u'450.00']]

In [27]:
clean = services.map(lambda line: line[1:] if line[0]=='#' else line)

In [28]:
clean

PythonRDD[16] at RDD at PythonRDD.scala:48

In [29]:
clean = clean.map(lambda line : line.split())

In [30]:
clean.collect()

[[u'EventId', u'Timestamp', u'Customer', u'State', u'ServiceID', u'Amount'],
 [u'201', u'10/13/2017', u'100', u'NY', u'131', u'100.00'],
 [u'204', u'10/18/2017', u'700', u'TX', u'129', u'450.00'],
 [u'202', u'10/15/2017', u'203', u'CA', u'121', u'200.00'],
 [u'206', u'10/19/2017', u'202', u'CA', u'131', u'500.00'],
 [u'203', u'10/17/2017', u'101', u'NY', u'173', u'750.00'],
 [u'205', u'10/19/2017', u'202', u'TX', u'121', u'200.00']]

In [31]:
clean.map(lambda lst : (lst[3],lst[-1])).collect()

[(u'State', u'Amount'),
 (u'NY', u'100.00'),
 (u'TX', u'450.00'),
 (u'CA', u'200.00'),
 (u'CA', u'500.00'),
 (u'NY', u'750.00'),
 (u'TX', u'200.00')]

In [32]:
pairs = clean.map(lambda lst : (lst[3],lst[-1]))

In [35]:
rekey = pairs.reduceByKey(lambda amt1,amt2 : float(amt1) + float(amt2))

In [36]:
rekey.collect()

[(u'State', u'Amount'), (u'CA', 700.0), (u'TX', 650.0), (u'NY', 850.0)]

In [38]:
clean.collect()

[[u'EventId', u'Timestamp', u'Customer', u'State', u'ServiceID', u'Amount'],
 [u'201', u'10/13/2017', u'100', u'NY', u'131', u'100.00'],
 [u'204', u'10/18/2017', u'700', u'TX', u'129', u'450.00'],
 [u'202', u'10/15/2017', u'203', u'CA', u'121', u'200.00'],
 [u'206', u'10/19/2017', u'202', u'CA', u'131', u'500.00'],
 [u'203', u'10/17/2017', u'101', u'NY', u'173', u'750.00'],
 [u'205', u'10/19/2017', u'202', u'TX', u'121', u'200.00']]

In [43]:
#grab (state,amount)
step1 = clean.map(lambda lst : (lst[3],lst[-1]))
# reduce by key
step2 = step1.reduceByKey(lambda amt1,amt2 : float(amt1) + float(amt2))
# get rid of state, amount titles
step3 = step2.filter(lambda x : not x[0] == 'State')
#sort results by amount
step4 = step3.sortBy(lambda stAmount : stAmount[1],ascending=False)
step4.collect()

[(u'NY', 850.0), (u'CA', 700.0), (u'TX', 650.0)]

In [44]:
x = ['ID','State', 'Amount']

In [45]:
def func1(lst):
    return lst[-1]

In [ ]:
def func2(id_st_amt):
    #unpack values
    ()